In [9]:
import webbrowser
from itertools import product

In [1]:
experiment_id_list = ["historical","ssp119","ssp126","ssp245","ssp370","ssp585"]
# variable_id_list = ["tas","tasmax","tasmin"]
variable_id_list = ["pr"]

nominal_resolution = ['50+km','100+km']
frequency_list = ['day','month']

In [2]:
# link_format = "https://aims2.llnl.gov/search?project=CMIP6&resultType=originals+only&activeFacets=%7B%22experiment_id%22%3A%22"+ experiment_id + "%22%2C%22variable_id%22%3A%22"+variable_id +"%22%2C%22nominal_resolution%22%3A%22"+nominal_resolution +"%22%2C%22frequency%22%3A%22"+frequency +"%22%7D"

NameError: name 'experiment_id' is not defined

In [3]:

# # Define the lists of variables
# experiment_id_list = ["historical", "ssp119", "ssp126", "ssp245", "ssp370", "ssp585"]
# variable_id_list = ["pr"]
# #                     , "tasmax", "tasmin"]
# nominal_resolution_list = ['50+km', '100+km']
# frequency_list = ['day', 'month']

# # Define the base URL format
# link_format = (
#     "https://aims2.llnl.gov/search?project=CMIP6&resultType=originals+only&activeFacets="
#     "%7B%22experiment_id%22%3A%22{experiment_id}%22%2C%22variable_id%22%3A%22{variable_id}%22%2C"
#     "%22nominal_resolution%22%3A%22{nominal_resolution}%22%2C%22frequency%22%3A%22{frequency}%22%7D"
# )

# # Generate all combinations of the variables
# combinations = product(experiment_id_list, variable_id_list, nominal_resolution_list, frequency_list)

# # Generate the URLs
# urls = [
#     link_format.format(
#         experiment_id=experiment_id,
#         variable_id=variable_id,
#         nominal_resolution=nominal_resolution,
#         frequency=frequency
#     )
#     for experiment_id, variable_id, nominal_resolution, frequency in combinations
# ]

# # Print the URLs
# for url in urls:
#     print(url)


NameError: name 'product' is not defined

In [17]:
from itertools import product

# Define the lists of variables
experiment_id_list = ["historical", "ssp119", "ssp126", "ssp245", "ssp370", "ssp585"]
variable_id_list = ["pr"]
nominal_resolution_list = ['50+km', '100+km']
frequency_list = ['day', 'month']

# Define the base URL format for JSON
json_link_format = (
    "https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset"
    "&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,"
    "source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,"
    "table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*"
    "&experiment_id={experiment_id}&frequency={frequency}&nominal_resolution={nominal_resolution}&variable_id={variable_id}"
)

# Generate all combinations of the variables
combinations = product(experiment_id_list, variable_id_list, nominal_resolution_list, frequency_list)

# Generate the JSON URLs
json_urls = [
    json_link_format.format(
        experiment_id=experiment_id,
        variable_id=variable_id,
        nominal_resolution=nominal_resolution.replace('+', '%20'),
        frequency=frequency
    )
    for experiment_id, variable_id, nominal_resolution, frequency in combinations
]

# Print the generated URLs
for url in json_urls:
    print(url)


https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=historical&frequency=day&nominal_resolution=50%20km&variable_id=tas
https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=historical&frequency=month&nominal_resolution=50%20km&variable_id=tas
https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facet

In [12]:
for url in urls:
    webbrowser.open(url)

In [13]:
import requests
import json
import os

# Define a directory to save downloaded files
download_dir = "./test"
os.makedirs(download_dir, exist_ok=True)

print("Querying ESGF URLs...")
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # Extract dataset file URLs
        if 'response' in data and 'docs' in data['response']:
            docs = data['response']['docs']
            for doc in docs:
                # Extract file URL (example: 'url' field in the JSON)
                if 'url' in doc:
                    file_urls = [u.split('|')[0] for u in doc['url']]
                    for file_url in file_urls:
                        print(f"Downloading: {file_url}")
                        file_response = requests.get(file_url, stream=True)
                        if file_response.status_code == 200:
                            filename = os.path.join(download_dir, os.path.basename(file_url))
                            with open(filename, 'wb') as f:
                                for chunk in file_response.iter_content(chunk_size=1024):
                                    f.write(chunk)
                        else:
                            print(f"Failed to download: {file_url}")
    else:
        print(f"Failed: {url} with status code {response.status_code}")


Querying ESGF URLs...


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
for url in json_urls:
    print(f"Querying URL: {url}")
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            # Process JSON response
            if 'response' in data and 'docs' in data['response']:
                docs = data['response']['docs']
                for doc in docs:
                    if 'url' in doc:
                        file_urls = [u.split('|')[0] for u in doc['url']]
                        for file_url in file_urls:
                            print(f"File URL: {file_url}")
        except requests.JSONDecodeError:
            print("JSONDecodeError: Could not parse response as JSON.")
            print(f"Response Text: {response.text}")
    else:
        print(f"Error: Received status code {response.status_code}")


Querying URL: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=historical&frequency=day&nominal_resolution=50%20km&variable_id=tas
File URL: http://esg1.umr-cnrm.fr/thredds/catalog/esgcet/188/CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1-HR.historical.r1i1p1f2.day.tas.gr.v20191021.xml#CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1-HR.historical.r1i1p1f2.day.tas.gr.v20191021
File URL: http://esg1.umr-cnrm.fr/las/getUI.do?catid=7ACBAEA5BFAB598DEEACC7E5DB49CE60_ns_CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1-HR.historical.r1i1p1f2.day.tas.gr.v20191021
Querying URL: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data

Querying URL: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=ssp126&frequency=day&nominal_resolution=50%20km&variable_id=tas
File URL: http://esg1.umr-cnrm.fr/thredds/catalog/esgcet/207/CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1-HR.ssp126.r1i1p1f2.day.tas.gr.v20200127.xml#CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1-HR.ssp126.r1i1p1f2.day.tas.gr.v20200127
File URL: http://esg1.umr-cnrm.fr/las/getUI.do?catid=7ACBAEA5BFAB598DEEACC7E5DB49CE60_ns_CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1-HR.ssp126.r1i1p1f2.day.tas.gr.v20200127
Querying URL: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id

Querying URL: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=ssp370&frequency=day&nominal_resolution=50%20km&variable_id=tas
File URL: http://esg1.umr-cnrm.fr/thredds/catalog/esgcet/208/CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1-HR.ssp370.r1i1p1f2.day.tas.gr.v20200127.xml#CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1-HR.ssp370.r1i1p1f2.day.tas.gr.v20200127
File URL: http://esg1.umr-cnrm.fr/las/getUI.do?catid=7ACBAEA5BFAB598DEEACC7E5DB49CE60_ns_CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1-HR.ssp370.r1i1p1f2.day.tas.gr.v20200127
Querying URL: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id

In [19]:
import requests

# Define the JSON URL
json_url = "https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id%2C+data_node%2C+source_id%2C+institution_id%2C+source_type%2C+experiment_id%2C+sub_experiment_id%2C+nominal_resolution%2C+variant_label%2C+grid_label%2C+table_id%2C+frequency%2C+realm%2C+variable_id%2C+cf_standard_name&latest=true&replica=false&query=*&experiment_id=historical&frequency=day&nominal_resolution=50%20km&variable_id=tas"

# Fetch the JSON response
response = requests.get(json_url)
if response.status_code != 200:
    print(f"Failed to fetch JSON data. Status code: {response.status_code}")
    exit()

# Parse the JSON response
data = response.json()
docs = data.get("response", {}).get("docs", [])

# Generate wget script
wget_script = []
for doc in docs:
    file_urls = doc.get("url", [])
    for file_url in file_urls:
        # Ensure only HTTP(S) URLs are used for wget
        if file_url.startswith("http"):
            wget_script.append(f"wget --no-check-certificate '{file_url}'")

# Save the wget script to a file
script_filename = "download_wget_script.sh"
with open(script_filename, "w") as f:
    f.write("\n".join(wget_script))

print(f"Wget script generated: {script_filename}")


Wget script generated: download_wget_script.sh


In [20]:
import requests

# Define the JSON URL
json_url = "https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id%2C+data_node%2C+source_id%2C+institution_id%2C+source_type%2C+experiment_id%2C+sub_experiment_id%2C+nominal_resolution%2C+variant_label%2C+grid_label%2C+table_id%2C+frequency%2C+realm%2C+variable_id%2C+cf_standard_name&latest=true&replica=false&query=*&experiment_id=historical&frequency=day&nominal_resolution=50%20km&variable_id=tas"

# Fetch the JSON response
response = requests.get(json_url)
if response.status_code != 200:
    print(f"Failed to fetch JSON data. Status code: {response.status_code}")
    exit()

# Parse the JSON response
data = response.json()
docs = data.get("response", {}).get("docs", [])

# Generate wget commands
wget_commands = []
for doc in docs:
    urls = doc.get("url", [])
    for url in urls:
        # Look for "HTTPServer" links to .nc files
        if "HTTPServer" in url and url.endswith(".nc"):
            wget_commands.append(f"wget --no-check-certificate '{url}'")

# Save the commands to a .sh file
script_filename = "wget_script_generated.sh"
with open(script_filename, "w") as f:
    f.write("#!/bin/bash\n")
    f.write("\n".join(wget_commands))

print(f"Download script generated: {script_filename}")


Download script generated: wget_script_generated.sh


In [21]:
import requests
from itertools import product

# Define the lists of variables
experiment_id_list = ["historical", "ssp119", "ssp126", "ssp245", "ssp370", "ssp585"]
variable_id_list = ["tas"]
nominal_resolution_list = ['50+km', '100+km']
frequency_list = ['day', 'month']

# Define the base API URLs
search_url = "https://esgf-node.llnl.gov/esg-search/search"
wget_script_url = "https://esgf-node.llnl.gov/esg-search/wget"

# Define the base URL format for JSON
json_link_format = (
    "https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset"
    "&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,"
    "source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,"
    "table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*"
    "&experiment_id={experiment_id}&frequency={frequency}&nominal_resolution={nominal_resolution}&variable_id={variable_id}"
)

# Generate all combinations of the variables
combinations = product(experiment_id_list, variable_id_list, nominal_resolution_list, frequency_list)

# Generate the JSON URLs
json_urls = [
    json_link_format.format(
        experiment_id=experiment_id,
        variable_id=variable_id,
        nominal_resolution=nominal_resolution.replace('+', '%20'),
        frequency=frequency
    )
    for experiment_id, variable_id, nominal_resolution, frequency in combinations
]

# Iterate over each JSON URL and process datasets
for json_url in json_urls:
    print(f"Querying: {json_url}")

    # Query the ESGF search API
    response = requests.get(json_url)
    if response.status_code != 200:
        print(f"Failed to fetch JSON data from {json_url}. Status code: {response.status_code}")
        continue

    # Parse the search results
    data = response.json()
    docs = data.get("response", {}).get("docs", [])

    # Generate wget scripts for each dataset
    for doc in docs:
        dataset_id = doc.get("id")
        if not dataset_id:
            print("Dataset ID not found in the response.")
            continue

        # Query the wget script API
        wget_response = requests.get(wget_script_url, params={"dataset_id": dataset_id, "project": "CMIP6"})
        if wget_response.status_code != 200:
            print(f"Failed to fetch wget script for dataset ID {dataset_id}. Status code: {wget_response.status_code}")
            continue

        # Save the wget script to a file
        script_filename = f"wget_script_{dataset_id.replace(':', '_').replace('/', '_')}.sh"
        with open(script_filename, "w") as f:
            f.write(wget_response.text)

        print(f"Generated wget script: {script_filename}")


Querying: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=historical&frequency=day&nominal_resolution=50%20km&variable_id=tas
Generated wget script: wget_script_CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1-HR.historical.r1i1p1f2.day.tas.gr.v20191021|esg1.umr-cnrm.fr.sh
Querying: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=historical&frequency=month&nominal_resolutio

KeyboardInterrupt: 

In [23]:
import os
import requests
from itertools import product

# Define the lists of variables
experiment_id_list = ["historical", "ssp119", "ssp126", "ssp245", "ssp370", "ssp585"]
variable_id_list = ["tas"]
nominal_resolution_list = ['50+km', '100+km']
frequency_list = ['day', 'month']

# Define the base API URLs
search_url = "https://esgf-node.llnl.gov/esg-search/search"
wget_script_url = "https://esgf-node.llnl.gov/esg-search/wget"

# Define the base URL format for JSON
json_link_format = (
    "https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset"
    "&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,"
    "source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,"
    "table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*"
    "&experiment_id={experiment_id}&frequency={frequency}&nominal_resolution={nominal_resolution}&variable_id={variable_id}"
)

# Create base directory
base_dir = "wgetCMIP6"
os.makedirs(base_dir, exist_ok=True)

# Generate all combinations of the variables
combinations = product(experiment_id_list, variable_id_list, nominal_resolution_list, frequency_list)

# Generate the JSON URLs
json_urls = [
    json_link_format.format(
        experiment_id=experiment_id,
        variable_id=variable_id,
        nominal_resolution=nominal_resolution.replace('+', '%20'),
        frequency=frequency
    )
    for experiment_id, variable_id, nominal_resolution, frequency in combinations
]

# Iterate over each JSON URL and process datasets
for json_url in json_urls:
    print(f"Querying: {json_url}")

    # Query the ESGF search API
    response = requests.get(json_url)
    if response.status_code != 200:
        print(f"Failed to fetch JSON data from {json_url}. Status code: {response.status_code}")
        continue

    # Parse the search results
    data = response.json()
    docs = data.get("response", {}).get("docs", [])

    # Generate wget scripts for each dataset
    for doc in docs:
        dataset_id = doc.get("id")
        variable_id = doc.get("variable_id", ["unknown"])[0]
        experiment_id = doc.get("experiment_id", "unknown")[0]

        if not dataset_id:
            print("Dataset ID not found in the response.")
            continue

        # Create directories for the variable and experiment
        variable_dir = os.path.join(base_dir, variable_id)
        experiment_dir = os.path.join(variable_dir, experiment_id)
        os.makedirs(experiment_dir, exist_ok=True)

        # Query the wget script API
        wget_response = requests.get(wget_script_url, params={"dataset_id": dataset_id, "project": "CMIP6"})
        if wget_response.status_code != 200:
            print(f"Failed to fetch wget script for dataset ID {dataset_id}. Status code: {wget_response.status_code}")
            continue

        # Save the wget script to the appropriate folder
        script_filename = os.path.join(experiment_dir, f"wget_script_{dataset_id.replace(':', '_').replace('/', '_')}.sh")
        with open(script_filename, "w") as f:
            f.write(wget_response.text)

        print(f"Generated wget script: {script_filename}")


Querying: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=historical&frequency=day&nominal_resolution=50%20km&variable_id=tas
Generated wget script: wgetCMIP6/tas/historical/wget_script_CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1-HR.historical.r1i1p1f2.day.tas.gr.v20191021|esg1.umr-cnrm.fr.sh
Querying: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=historical&frequenc

Generated wget script: wgetCMIP6/tas/ssp126/wget_script_CMIP6.ScenarioMIP.BCC.BCC-CSM2-MR.ssp126.r1i1p1f1.day.tas.gn.v20190315|cmip.bcc.cma.cn.sh
Generated wget script: wgetCMIP6/tas/ssp126/wget_script_CMIP6.ScenarioMIP.NCAR.CESM2.ssp126.r11i1p1f1.day.tas.gn.v20200528|esgf-data.ucar.edu.sh
Generated wget script: wgetCMIP6/tas/ssp126/wget_script_CMIP6.ScenarioMIP.NCAR.CESM2.ssp126.r10i1p1f1.day.tas.gn.v20200528|esgf-data.ucar.edu.sh
Generated wget script: wgetCMIP6/tas/ssp126/wget_script_CMIP6.ScenarioMIP.NCAR.CESM2.ssp126.r4i1p1f1.day.tas.gn.v20200528|esgf-data.ucar.edu.sh
Generated wget script: wgetCMIP6/tas/ssp126/wget_script_CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp126.r3i1p1f1.day.tas.gn.v20210907|esgf-data03.diasjp.net.sh
Generated wget script: wgetCMIP6/tas/ssp126/wget_script_CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp126.r4i1p1f1.day.tas.gn.v20210907|esgf-data03.diasjp.net.sh
Generated wget script: wgetCMIP6/tas/ssp126/wget_script_CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp126.r5i1p1f1.day.tas.gn.

Querying: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=ssp585&frequency=day&nominal_resolution=50%20km&variable_id=tas
Generated wget script: wgetCMIP6/tas/ssp585/wget_script_CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1-HR.ssp585.r1i1p1f2.day.tas.gr.v20191202|esg1.umr-cnrm.fr.sh
Querying: https://esgf-node.llnl.gov/esg-search/search?project=CMIP6&offset=0&limit=10&type=Dataset&format=application%2Fsolr%2Bjson&facets=activity_id,data_node,source_id,institution_id,source_type,experiment_id,sub_experiment_id,nominal_resolution,variant_label,grid_label,table_id,frequency,realm,variable_id,cf_standard_name&latest=true&replica=false&query=*&experiment_id=ssp585&frequency=month&n